In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import os, sys, fnmatch
import numpy as np
from lxml import etree
import hashlib, json
import re
import pandas as pd
import pickle
from tqdm import tqdm_notebook
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
with open('./public_bouy_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
with open('./public_bouy_target.pkl', 'rb') as f:
    targets = pickle.load(f)

In [7]:
history_length = 30

In [8]:
def get_Classification_problem_tables(df, target_array, history_length):
    X_arrays = []
    Y_values = []
    for start in range(df.shape[0]):
        if start < history_length-1:
            continue
        if start+forecast_range >= df.shape[0]:
            break
            
        x = df[['dx','dy','dz']].iloc[start+1-history_length:start+1]
        X_arrays.append(x)
        
        y = target_array[start]
        Y_values.append(y)
    
    return X_arrays, Y_values

In [9]:
def vectorize_data(Xdata, Ydata):
    retX = []
    retY = []
    for x,y in zip(Xdata, Ydata):
        xprime = x.values.reshape((-1,1))
        yprime = y.reshape((-1,1))
        retX.append(xprime)
        retY.append(yprime)
    return retX, retY

In [10]:
x_AR = []
y_AR = []
for d,target_array in tqdm_notebook(zip(data, targets), total=len(data)):
    x, y = get_Classification_problem_tables(d, target_array, history_length)
    x_AR.append(x)
    y_AR.append(y)

/tmp/ipykernel_23270/2541459539.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for d,target_array in tqdm_notebook(zip(data, targets), total=len(data)):


  0%|          | 0/40 [00:00<?, ?it/s]

In [15]:
with open('./X_classif_vectorized.pkl', 'rb') as f:
    xVectorized_all = pickle.load(f)

In [16]:
with open('./Y_classif_vectorized.pkl', 'rb') as f:
    yVectorized_all = pickle.load(f)

In [21]:
with open('./X_classif_vectorized_concat.pkl', 'rb') as f:
    xVectorized_concat = pickle.load(f)

In [22]:
with open('./Y_classif_vectorized_concat.pkl', 'rb') as f:
    yVectorized_concat = pickle.load(f)

In [29]:
x_train, x_test, y_train, y_test = train_test_split(xVectorized_concat, yVectorized_concat, test_size=0.25)

In [32]:
m = RandomForestClassifier(n_jobs=20, verbose=2)

In [33]:
m.fit(x_train, y_train.ravel())

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100


[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    6.1s


building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63

[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:   47.8s finished


RandomForestClassifier(n_jobs=20, verbose=2)

In [34]:
m.verbose=0

In [35]:
y_test_pred = m.predict(x_test)

In [37]:
y_test.shape

(58321, 1)

In [38]:
y_test_pred.shape

(58321,)

In [39]:
y_test_pred.sum()

0

In [36]:
f1_score(y_test.ravel(), y_test_pred)

0.0